In [13]:
import numpy as np
import random

#問題読み取り関数
def puzzle_generation(puzzle_board, issue):
    #ファイル読み込み
    f = open(issue, 'r', encoding='UTF-8')
    data = f.readlines()
    f.close()

    #問題フォーマットの1行目の読み取り
    s0 = data[0].rstrip('\n').rstrip(',')
    l0 = [int(x.strip()) for x in s0.split(',')]
    column = l0[0]              #列
    row = l0[1]                 #行
    selection_cost = l0[2]      #選択コスト

    #スライドパズルの盤面の読み取り
    for i in range(1, row+1):
        row_data = data[i].rstrip('\n').rstrip(',')
        line_data = [int(x.strip()) for x in row_data.split(',')]
        row_array = np.array(line_data)
        if i == 1:
            puzzle_board = row_array
        else:
            puzzle_board = np.vstack((puzzle_board, row_array))

    return puzzle_board, column, row, selection_cost


#答え書き込み関数
def answer_writing(selection_position, exchange_operation):
    f = open('answer.txt', 'a')
    f.write(str(selection_position[1]) + str(selection_position[0]) + '\n')
    f.write(str(len(exchange_operation)) + '\n')
    f.write(exchange_operation + '\n')
    f.close()

    exchange_operation = ""

    return exchange_operation


#移動関数
#選択値を上に移動
def up(target_value, selection_value, exchange_operation):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]-1][selection_coord[1]])
    np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]-column], selection_value)

    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    exchange_operation += 'U'

    return target_coord, selection_coord, exchange_operation

#選択値を下に移動
def down(target_value, selection_value, exchange_operation):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))


    np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]+1][selection_coord[1]])
    np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]+column], selection_value)

    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    exchange_operation += 'D'

    return target_coord, selection_coord, exchange_operation

#選択値を左に移動
def left(target_value, selection_value, exchange_operation):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]][selection_coord[1]-1])
    np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]-1], selection_value)

    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    exchange_operation += 'L'

    return target_coord, selection_coord, exchange_operation

#選択値を右に移動
def right(target_value, selection_value, exchange_operation):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]][selection_coord[1]+1])
    np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]+1], selection_value)

    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    exchange_operation += 'R'

    return target_coord, selection_coord, exchange_operation


#並び替え処理
# 2行を除いた盤面の並び替え(たくと)
def puzzle_board_sort_top(puzzle_board, selection_times, exchange_operation):
    #選択値の初期位置を記憶させておく
    selection_value = column * row
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))
    selection_position = selection_coord                #選択値初期位置

    #移動処理
    for i in range(row - 2):
        for j in range(column):
            target_value = i * column + j + 1           #動かしたい値
            selection_value = column * row              #選択値

            target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
            selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

            #最終的に置きたい座標
            if target_value % column == 0: coord = (target_value // column - 1, column - 1)
            else: coord = (target_value // column, target_value % column - 1)


            #targetが下端でない
            if target_coord[0] != row-1:
                #最終的に起きたい位置が右2列以外
                if j != column-1 and j != column-2:

                    #targetが最終的に置きたいx座標よりも右
                    if target_coord[1] > coord[1]:
                        #targetを最終的に置きたいy座標に移動
                        if target_coord[0] < selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        else:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                        #targetの左下にselectionを置く
                        if target_coord[1] <= selection_coord[1]:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        else:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                        #targetを置きたい座標のx座標と同じになるまで移動
                        if target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            while target_coord[1] != coord[1]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                    #targetを最終的に置きたいx座標よりも左
                    elif target_coord[1] < coord[1]:
                        if target_coord[0] < selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        elif target_coord[0] >= selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                        #targetの右下にselectionを置く
                        if target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)

                        #targetを置きたい座標のx座標と同じになるまで移動
                        if target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                            while target_coord[1] != coord[1]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                            #targetの右にselectionをおく
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

                    #targetが最終的に置きたいx座標と同じ
                    elif target_coord[1] == coord[1]:
                        #targetの右にselectionをおく
                        #targetよりselectionが右
                        if target_coord[1] < selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                            #targetよりselectionが上
                            if target_coord[0] > selection_coord[0]:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            #targetよりselectionが下
                            else:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

                        #targetよりselectionが左
                        elif target_coord[1] > selection_coord[1]:
                            #targetがselectionより上or下
                            if target_coord[0] != selection_coord[0]:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                #targetよりselectionが上
                                if target_coord[0] > selection_coord[0]:
                                    while target_coord[0] != selection_coord[0]:
                                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                                #targetよりselectionが下
                                else:
                                    while target_coord[0] != selection_coord[0]:
                                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            #targetがselectionと同じx座標
                            else:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

                        #targetとselectionのx座標が同じ
                        else:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            #targetよりselectionが上
                            if target_coord[0] > selection_coord[0]:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            #targetよりselectionが下
                            else:
                                while target_coord[0] != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

                    #targetを置きたい座標のy座標と同じになるまで移動
                    if target_coord[0] != coord[0]:
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        while target_coord[0] != coord[0]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                #最終的に置きたい位置が右から2列目
                elif j == column-2:
                    #最終的に置きたい座標にある
                    if target_coord[0] == coord[0] and target_coord[1] == coord[1]:
                        while target_coord[1]+1 != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        while target_coord[0] != selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)


                    #最終的に置きたいy座標でない
                    elif target_coord[0] != coord[0]:
                        #最終的におきたいx座標でない
                        if target_coord[1] != coord[1]+1:
                            #targetよりも下にselectionがある
                            if target_coord[0] < selection_coord[0]:
                                while target_coord[0]+1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            #targetよりも上にselectionがある
                            elif target_coord[0] >= selection_coord[0]:
                                while target_coord[0]+1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                            #targetの右にselectionを移動
                            #targetよりも右にselectionがある
                            if target_coord[1] < selection_coord[1]:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                            #targetよりも左にselectionがある
                            else:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                            #targetを右端まで移動
                            if target_coord[1] != coord[1]+1:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                                while target_coord[1] != coord[1]+1:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)

                            #targetを最終的に置きたいy座標まで移動
                            if target_coord[0] != coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                                while target_coord[0] != coord[0]:
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                        #最終的におきたいx座標にある
                        elif target_coord[1] == coord[1]+1:
                            #targetのx座標の左にselectionを移動
                            if target_coord[1] == selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                            else:
                                while target_coord[1]-1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                            #targetの左上に移動
                            #targetよりも下にselectionがある
                            if target_coord[0] <= selection_coord[0]:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            #targetよりも上にselectionがある
                            else:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                            #targetを最終的に置きたいy座標まで移動
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            while target_coord[0] != coord[0]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                #最終的に置きたい位置が右端
                else:
                    #逆順に並んでいる
                    if target_coord[0] == coord[0] and target_coord[1] == coord[1]-1:
                        #targetの下にselectionを移動
                        if target_coord[1] < selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                    elif target_coord[0]-1 == selection_coord[0] and target_coord[1] == selection_coord[1] and selection_coord[0] == coord[0]:
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                    #targetがtarget-1
                    elif target_coord[0] == coord[0]+1 and target_coord[1] == coord[1]:
                        #targetの左にselection
                        #targetの左の座標に移動
                        if target_coord[1] > selection_coord[1]:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        else:
                            while target_coord[1]-1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        #targetの左上に移動
                        if target_coord[0] <= selection_coord[0]:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

                        #3,4を最終的に置きたい座標となるよう移動
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                    elif target_coord[1] == coord[1]:
                        #targetの上にselectionがある
                        if target_coord[0] > selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        #targetの下にselectionがある
                        elif target_coord[0] < selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                        while target_coord[0] != coord[0]:
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                    elif target_coord[0] != coord[0]+1 or target_coord[1] != coord[1]:
                        #targetの上にselectionがある
                        if target_coord[0] >= selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        #targetの下にselectionがある
                        elif target_coord[0] < selection_coord[0]:
                            while target_coord[0]+1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation) 

                        #targetの右にselectionがある
                        if target_coord[1] < selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        #targetの左にselectionがある
                        elif target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                        #targetを右端まで移動
                        # if target_coord[1] != coord[1]:
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        while target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)

                        #targetをtarget-1の下に移動
                        if target_coord[0] != coord[0]+1:
                            while target_coord[0] != coord[0]+1:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

                        #3,4を最終的に置きたい座標となるよう移動
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)


            #targetが下端
            else:
                #最終的に起きたい位置が右2列以外
                if j != column-1 and j != column-2:
                    #右2列を除いた列を並び替え
                    if j != row - 1 and j != row - 2:
                        #targetが最終的におきたいx座標よりも右
                        if target_coord[1] > coord[1]:
                            #targetの上の座標にselectionを置く
                            if target_coord[0] == selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            else:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            #targetの左上にselectionを置く
                            if target_coord[1] <= selection_coord[1]:
                                while target_coord[1]-1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                            else:
                                while target_coord[1]-1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                            #targetをおきたい座標のx座標と同じになるまで移動
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            while target_coord[1] != coord[1]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            #targetの上にselectionを置く
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)

                        #targetが最終的におきたいx座標よりも左
                        elif target_coord[1] < coord[1]:
                            #targetの上の座標にselectionを置く
                            if target_coord[0] == selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            else:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            #targetの右上にselectionを置く
                            if target_coord[1] >= selection_coord[1]:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            else:
                                while target_coord[1]+1 != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)

                            #targetを置きたい座標のx座標と同じになるまで移動
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                            while target_coord[1] != coord[1]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)

                            #targetの上にchoicを置く
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                        #targetが最終的におきたいx座標と同じ
                        else:
                            #targetの上の座標にselectionを置く
                            if target_coord[0] == selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            else:
                                while target_coord[0]-1 != selection_coord[0]:
                                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                            #targetの上にselectionを置く
                            if target_coord[1] < selection_coord[1]:
                                while target_coord[1] != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                            elif target_coord[1] > selection_coord[1]:
                                while target_coord[1] != selection_coord[1]:
                                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                        #targetを置きたい座標のy座標になるまで移動
                        if target_coord[0] != coord[0]:
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            while target_coord[0] != coord[0]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                #最終的に置きたい位置が右から2列目
                elif j == column-2:
                    #targetが最終的におきたいx座標よりも左
                    if target_coord[1] < coord[1]+1:
                        #targetの上の座標にselectionを置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        #targetの右上にselectionを置く
                        if target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)

                        #targetを置きたい座標のx座標と同じになるまで移動
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        while target_coord[1] != coord[1]+1:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                    #targetが最終的におきたいx座標の右と同じ
                    elif target_coord[1] == coord[1]+1:
                        #targetの上の座標にselectionを置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        #targetの上にselectionを移動
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                    #targetを最終的に置きたいy座標まで移動
                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                    while target_coord[0] != coord[0]:
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                #最終的に置きたい位置が右端
                else:
                    #targetが最終的におきたいx座標よりも左
                    if target_coord[1] < coord[1]:
                        #targetの上の座標にselectionを置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        #targetの右上にselectionを置く
                        if target_coord[1] >= selection_coord[1]:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)

                        #targetを置きたい座標のx座標と同じになるまで移動
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        while target_coord[1] != coord[1]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                    #targetが最終的におきたいx座標と同じ
                    elif target_coord[1] == coord[1]:
                        #targetの上の座標にselectionを置く
                        if target_coord[0] == selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        else:
                            while target_coord[0]-1 != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        #targetの上にselectionを移動
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                    #targetを最終的に置きたいy座標まで移動
                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                    while target_coord[0] != coord[0]+1:
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

                    #3,4を最終的におきたい座標となるよう移動
                    target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                    target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

    return exchange_operation, selection_position


# 下2行の右4マスを除いた盤面の並び替え(たいち)
def puzzle_board_sort_bottom_left(puzzle_board, selection_times, exchange_operation):
    #target_valueの求めかた
    y = 8       #行
    x = 8       #列
    for i in range(x-2):
        for j in reversed(range(y-2, y)):
            target_value = j * x + i+1
            selection_value = row * column

            target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
            selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

            #一旦置きたい座標(4*4)
            #if target_value == 13: coord = (2, 0)
            #elif target_value == 9: coord = (2, 1)
            #elif target_value == 14: coord = (2, 1)
            #elif target_value == 10: coord = (2, 2)

            #一旦起きたい座標(6*6)
            # if target_value == 31: coord = (4, 0)
            # elif target_value == 25: coord = (4, 1)
            # elif target_value == 32: coord = (4, 1)
            # elif target_value == 26: coord = (4, 2)
            # elif target_value == 33: coord = (4, 2)
            # elif target_value == 27: coord = (4, 3)
            # elif target_value == 34: coord = (4, 3)
            # elif target_value == 28: coord = (4, 4)

            #一旦置きたい座標(8*8)
            if target_value == 57: coord = (6, 0)
            elif target_value == 49: coord = (6, 1)
            elif target_value == 58: coord = (6, 1)
            elif target_value == 50: coord = (6, 2)
            elif target_value == 59: coord = (6, 2)
            elif target_value == 51: coord = (6, 3)
            elif target_value == 60: coord = (6, 3)
            elif target_value == 52: coord = (6, 4)
            elif target_value == 61: coord = (6, 4)
            elif target_value == 53: coord = (6, 5)
            elif target_value == 62: coord = (6, 5)
            elif target_value == 54: coord = (6, 6)

            #一旦置きたい座標のy座標を揃える
            if target_coord[0] != coord[0]:
                #targetよりchoiceの方が上
                if target_coord[0] > selection_coord[0]:
                    #targetよりchoiceの方が右
                    if target_coord[1] < selection_coord[1]:
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                    #targetよりchoiceの方が左
                    elif target_coord[1] > selection_coord[1]:
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                #targetとchoiceのy座標が同じ
                else:
                    target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                    #targetよりchoiceの方が右
                    if target_coord[1] < selection_coord[1]:
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                    #targetよりchoiceの方が左
                    elif target_coord[1] > selection_coord[1]:
                        while target_coord[1] != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                #targetのy座標をcoordと揃える
                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

            #targetが一旦置きたいx座標よりも右
            if target_coord[1] > coord[1]:
                #targetの左にchoiceを置く
                if target_coord[1] <= selection_coord[1]:
                    while target_coord[1]-1 != selection_coord[1]:
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                else:
                    while target_coord[1]-1 != selection_coord[1]:
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                if target_coord[0] < selection_coord[0]:
                    while target_coord[0] != selection_coord[0]:
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

                #targetを置きたい座標のx座標と同じになるまで移動
                if target_coord[1] != coord[1]:
                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

                    while target_coord[1] != coord[1]:
                        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

            #targetが一旦置きたいx座標よりも左
            elif target_coord[1] < coord[1]:
                #targetの右にchoiceを置く
                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

            #targetが最終的に置きたいx座標と同じ
            elif target_coord[1] == coord[1]:
                #targetの右にchoiceをおく
                #targetよりchoiceが右
                if target_coord[1] < selection_coord[1]:
                    while target_coord[1]+1 != selection_coord[1]:
                        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
                    #targetよりchoiceが下
                    if target_coord[0] < selection_coord[0]:
                        while target_coord[0] != selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

                #targetよりchoiceが左
                elif target_coord[1] > selection_coord[1]:
                    #targetがchoiceより上or下
                    if target_coord[0] != selection_coord[0]:
                        while target_coord[1]+1 != selection_coord[1]:
                            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                        #targetよりchoiceが下
                        if target_coord[0] < selection_coord[0]:
                            while target_coord[0] != selection_coord[0]:
                                target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
                        #targetがchoiceと同じx座標
                        else:
                            while target_coord[1]+1 != selection_coord[1]:
                                target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

                #targetとchoiceのx座標が同じ
                else:
                    target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
                    #targetよりchoiceが下
                    if target_coord[0] < selection_coord[0]:
                        while target_coord[0] != selection_coord[0]:
                            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

        #右回りして値を格納する
        target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
        target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
        target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

    return exchange_operation


# 右下4ますの並び替え(ゆうすけ)
def puzzle_board_sort_bottom_right(puzzle_board, selection_times, selection_position, exchange_operation):
    target_value = column * row - row - 1
    selection_value = column * row

    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))
    selection_position = selection_coord

    #左上にすでに置きたい数字があるとき
    if target_coord == (column - 2, row - 2):
        target_value = column * row - row
        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))

        #右上にすで置きたい数字があるとき
        if target_coord == (column - 2, row - 1):
            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation) 

        #右上に置きたい数字がないとき
        else:
            selection_value = column * row - 1
            selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))
            selection_position = selection_coord

            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)

        exchange_operation = answer_writing(selection_position, exchange_operation)

    #左上に置きたい数字がないとき
    else:
        #choiceを左上に移動
        if selection_coord != (column - 2, row - 2):
            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

        #choiceを右に移動
        target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

        #左上に置きたい数字が来るまで時計回りで回転
        while target_coord != (column - 2, row - 2):
            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)
            target_coord, selection_coord, exchange_operation = left(target_value, selection_value, exchange_operation)
            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)
            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)

        #残りの処理
        #左下に置きたい数字ををターゲット
        target_value = column * row - 1
        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))

        #左下にすでに置きたい数字があるとき
        if target_coord == (column - 1, row - 2):
            target_coord, selection_coord, exchange_operation = down(target_value, selection_value, exchange_operation)

            exchange_operation = answer_writing(selection_position, exchange_operation)

        #左下に置きたい数字がないとき
        else:
            target_value = column * row
            selection_value = column * row - row

            target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
            selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))
            selection_position = selection_coord


            target_coord, selection_coord, exchange_operation = right(target_value, selection_value, exchange_operation)
            target_coord, selection_coord, exchange_operation = up(target_value, selection_value, exchange_operation)

            exchange_operation = answer_writing(selection_position, exchange_operation)

    return exchange_operation, selection_position


#分担した処理をまとめる
def puzzle_board_sort(puzzle_board):
    selection_times = 0                     #選択回数
    exchange_operation = ""                 #交換操作

    #回答 選択回数書き込み
    f = open('answer.txt', 'w')
    f.write('2\n')
    f.close()

    #たくと担当
    exchange_operation, selection_position = puzzle_board_sort_top(puzzle_board, selection_times, exchange_operation)

    #たいち担当
    exchange_operation = puzzle_board_sort_bottom_left(puzzle_board, selection_times, exchange_operation)
    exchange_operation = answer_writing(selection_position, exchange_operation)

    # ゆうすけ担当
    exchange_operation, selection_position = puzzle_board_sort_bottom_right(puzzle_board, selection_times, selection_position, exchange_operation)


#メイン
puzzle_board =  np.array([])    #現在の盤面を表す配列

#問題盤面生成
puzzle_board, column, row, selection_cost = puzzle_generation(puzzle_board, 'issue/T8x8.txt')

print('column(列→)：' + str(column))
print('row(行↓)：' + str(row))
print('selection_cost(選択コスト)：' + str(selection_cost) + '\n')
print('↓入力盤面')
print(puzzle_board)
print()

# 並び替え
puzzle_board_sort(puzzle_board)

print()
print('↓並び替え完了!!!!!!!!')
print(puzzle_board)


column(列→)：8
row(行↓)：8
selection_cost(選択コスト)：16

↓入力盤面
[[61 49 40 60  6 19  1 20]
 [ 9 55 50 32 45 21 11 63]
 [17 12 54 41 18 58  4 25]
 [14  5 24 52 47 23  3 56]
 [35 29 15 22 30 62 39 34]
 [38 10 46 44 36 28 33  8]
 [26 37  7  2 16 43 48 13]
 [57 42 59 27 31 51 64 53]]


↓並び替え完了!!!!!!!!
[[ 1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16]
 [17 18 19 20 21 22 23 24]
 [25 26 27 28 29 30 31 32]
 [33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48]
 [49 50 51 52 53 54 55 56]
 [57 58 59 60 61 62 63 64]]
